In [1]:
import json
import xlwt
import time
import os
import re
from html import unescape
from googletrans import Translator

In [2]:
output_dir="output"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [3]:
now=lambda :time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())

In [4]:
def load_json(json_fname):
    with open(json_fname,encoding="utf8") as fp:
        return json.load(fp)

In [5]:
title_key=lambda title:re.sub("\W","",title.lower(),)

In [12]:
articles_fname="article_list.json"

# from EI result to articles_list.json

In [6]:
json18="CC219419-China&HK.json"
json16="CC184029-China&HK.json"
article_list=load_json(json16)["results"]+load_json(json18)["results"]

In [7]:
i=0
while i<len(article_list):
    title=article_list[i]["titleNoHighlight"].lower().strip()
    if len(title.split(" "))<3:
        print(title)
    if title in {"preface"}:
        article_list.pop(i)
    else:
        i+=1

preface
preface
video re-localization


In [8]:
article_ids={}
for article in article_list:
    c=0
    year=article["yr"]
    article_id=article_ids.setdefault(year,0)+1
    article_ids[year]=article_id
    article["myid"]=f"ECCV{year}{article_id:03}"

In [9]:
with open("affils_trans_map.json") as fp:
    affils_trans_map=json.load(fp)

In [10]:
links_map=load_json("links_map.json")
for article in article_list:
    title=article["titleNoHighlight"]
    link_d=links_map.get(title_key(title),{})
    article["full_text_link"]=link_d.get("link","")
    article["file_name"]=link_d.get("file_name","")

In [13]:
with open(articles_fname,"w") as fp:
    json.dump({"articles":article_list,"affils":affils_trans_map},fp,indent=2)

# from article_list.json to excel

In [14]:
translator=Translator(service_urls=["translate.google.cn"])
_trans=lambda source:translator.translate(source ,dest="zh-cn",src="en").text
def trans(source):
    if source not in trans.map:
        trans.map[source]=_trans(unescape(source))
        print(f"{source} -> {trans.map[source]}")
        with open(articles_fname,"w") as fp:
            json.dump(trans.json,fp)
    return unescape(trans.map[source])

with open(articles_fname) as fp:
    trans.json=json.load(fp)
    trans.map=trans.json["affils"]

In [15]:
# def text_link(title)->str:
#     return text_link.map.get(title.lower(),{}).get("link","")

# with open("links_map.json") as fp:
#     text_link.map=json.load(fp)

In [16]:
def text_link(title)->str:
    return text_link.map.get(title.lower(),{}).get("link","")

with open("links_map.json") as fp:
    text_link.map=json.load(fp)

In [17]:
article_list_dict=load_json(articles_fname)
article_list=article_list_dict["articles"]
# html_parser=HTMLParser()
wb=xlwt.Workbook(encoding="utf8")
ws=wb.add_sheet("论文及作者")

# write head_article
head_article=["编号","文章名称","发表时间","文件名","doi"]
# write head_authors
head_author=["作者姓名","邮箱","单位","单位-英文"]

width_list=[14,40,8,30,15
            ,15,15,30,30]
r=0
c=0
for head in head_article+head_author:
    ws.write(r,c,head)
    ws.col(c).width=width_list[c]*256
    c+=1
r+=1

# write article
for article in article_list:
    c=0
    year=article["yr"]
    article_id=article["myid"]
    title=unescape(article['titleNoHighlight'])
    
    for attr in [
        article["myid"],
        title,
        year,
        article["file_name"],
        article["doi"],
    ]:
        attr=unescape(attr)
        ws.write(r,c,attr)
        
        c+=1
#     ca=c
    for author in article["authors"]:
        name=author["name"]
        email=author["email"]
        affils_list=[unescape(aff["name"])  for aff in author["affils"]]
        affils="; ".join([trans(aff) for aff in affils_list])
        affils_en="; ".join(affils_list)
        ca=c
        for attr in [name,email,affils,affils_en]:
            attr=unescape(attr)
            ws.write(r,ca,attr)
            ca+=1
        r+=1
    
    
wb.save(os.path.join(output_dir,f"output{now()}.xls"))